In [186]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
from bs4 import BeautifulSoup
import pandas as pd

In [187]:
keywords = ['MATH', 'APMTH', 'STAT', 'ECON','BIOSTAT','BIOL', 'HEB', 'MCB','OEB', 'PHYSICS','ASTRON', 'BIOPHYS', 'CHEM', 'ESPP', 'LIFESCI','MCB','SCRB']
DEPARTMENT_KEYWORD = 'PHYSICS'

In [188]:
data_directory = os.getcwd()+'/all_syllabi/'
data_directory

'/Users/mafuangimemkamon/Documents/GitHub/harvard_syllabi_topic_model/all_syllabi/'

In [189]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : data_directory}
chromeOptions.add_experimental_option("prefs",prefs)

In [190]:
driver = webdriver.Chrome(options=chromeOptions)
driver.get("https://syllabus.harvard.edu/search/courses/?q={}&schoolFilter=colgsas".format(DEPARTMENT_KEYWORD))

In [191]:
def count_results(driver):
    result_text = page_html = driver.page_source
    match = re.search(r"Loaded (\d+) results", result_text)
    if match:
        number_of_results = int(match.group(1))
        return number_of_results
    else:
        return 0

In [192]:
# Click expand more button until can't click anymore
num = 0
new_num = count_results(driver)
while new_num > num:
    time.sleep(0.5)
    try:
        button = driver.find_element("css selector", "button.css-jrdf8z-view--inlineBlock-baseButton")
        button.click()
    except:
        pass
    time.sleep(1)
    num = new_num
    new_num = count_results(driver)
    print(new_num, num, end='\r')

KeyboardInterrupt: 

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
names = [i.text for i in soup.select('.css-32gbjx-text')][1:]
ids = [i.get('id') for i in soup.find_all('tr')][1:]
assert len(names) == len(ids)
print(len(names))

In [ ]:
sdf = pd.read_csv('name_mapping/ALL.csv')
adf = pd.DataFrame(names, columns = ['Name'])
adf['code'] = adf['Name'].str.split(':').str[0]

sdf = pd.concat([sdf, adf])
sdf = sdf.drop_duplicates()
sdf.to_csv('name_mapping/{}.csv'.format('ALL'), index = False)

In [ ]:
#sdf = pd.read_csv('name_mapping/APMTH.csv')
#sdf.to_csv('name_mapping/ALL.csv', index = False)

In [ ]:
for i,id in enumerate(ids):
    print(i, names[i])
    url = "https://syllabus.harvard.edu/search/courses/?c={}&q={}".format(id, DEPARTMENT_KEYWORD)
    driver.get(url)
    time.sleep(1.5)
    try:
        download_button = driver.find_element("css selector", "button#download-btn")
        download_button.click()
    except:
        pass
    time.sleep(1.5)

In [ ]:
driver.quit()

In [ ]:
sdf